In [1]:
# Loading the Google API key from a Text File
with open('apikey.txt') as f:
    api_key = f.readline()
    f.close

In [2]:
# Configuring the API key
import gmaps
gmaps.configure(api_key=api_key)

In [3]:
# Loading the Sites Data
import pandas as pd

sites = pd.read_csv('/Users/vivekchowdary/Desktop/Capstone Project/Sites.csv')
sites_lat_long = sites[['latitude','longitude']]

sites_layer = gmaps.symbol_layer(
sites_lat_long, fill_color='green', stroke_color='green'
)

In [4]:
###################### Selecting the Source Probe ID using a drop-down list! ############################
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout
from IPython.display import display
import functools
 
output_dataframe = None

# Get our unique values
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

output = widgets.Output()

# Dropdown listbox
dropdown_source = widgets.Dropdown(description='Source:',
                                  options = unique_sorted_values_plus_ALL(sites.Source))

        
        
# Function to filter out dropdown listbox
def common_filtering(source):
    global output_dataframe
    df = sites.copy()
        
    filters = []
    
    # Evaluate our dropdown listbox and return booleans for our selections
    if source is not ALL:
        filters.append(df['Source'] == source)
    
    output.clear_output()
    
    with output:
        if filters:
            df_filter = functools.reduce(lambda x,y: x&y, filters)
            output_dataframe = df.loc[df_filter]
        else:
            output_dataframe = df
        display(output_dataframe)

def dropdown_source_eventhandler(change):
    common_filtering(change.new)

dropdown_source.observe(dropdown_source_eventhandler, names='value')

ui = widgets.HBox([dropdown_source])

display(ui, output)

Output()

In [70]:
# Storing the Retrieved Source Probe ID Data in a DataFrame Variable
x = pd.DataFrame(output_dataframe)
x = x.reset_index(drop=True)

source = x['latitude'][0],x['longitude'][0]
source

(-35.005234, 138.557875)

In [71]:
###################### Selecting the Destination Probe ID using a drop-down list! ############################
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout
from IPython.display import display
import functools
 
output_dataframe = None

# Get our unique values
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

output = widgets.Output()

# Dropdown listbox
dropdown_destination = widgets.Dropdown(description='Destination:',
                                  options = unique_sorted_values_plus_ALL(sites.Destination))

        
        
# Function to filter our dropdown listboxe
def common_filtering(destination):
    global output_dataframe
    df = sites.copy()
        
    filters = []
    
    # Evaluate our dropdown listbox and return booleans for our selections
    if destination is not ALL:
        filters.append(df['Destination'] == destination)
    
    output.clear_output()
    
    with output:
        if filters:
            df_filter = functools.reduce(lambda x,y: x&y, filters)
            output_dataframe = df.loc[df_filter]
        else:
            output_dataframe = df
        display(output_dataframe)

def dropdown_destination_eventhandler(change):
    common_filtering(change.new)

dropdown_destination.observe(dropdown_destination_eventhandler, names='value')

ui = widgets.HBox([dropdown_destination])

display(ui, output)

Output()

In [72]:
# Storing the Retrieved Destination Probe ID Data in a DataFrame Variable
x = pd.DataFrame(output_dataframe)
x = x.reset_index(drop=True)

destination = x['latitude'][0],x['longitude'][0]
destination

(-34.919526, 138.494303)

In [73]:
#Create the map
fig = gmaps.figure()
#create the layer
layer1 = gmaps.directions.Directions(source, destination, mode='car')
#layer = gmaps.directions.Directions(source, destination, waypoints=[montreux], mode='car')
#Add the layer
#fig.add_layer(layer)
fig.add_layer(layer1)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# This code plots all the sites as a bubble plot (However, it is too slow as there are more than 1k points)
#fig.add_layer(sites_layer)
#fig

Figure(layout=FigureLayout(height='420px'))

In [74]:
# Selecting a date and time to retrieve the travel distance and time taken in Kms and Minutes respectively
from ipydatetime import DatetimePicker

A = DatetimePicker()
A

DatetimePicker(value=None)

In [75]:
out = A.value
out

datetime.datetime(2021, 11, 8, 9, 38, tzinfo=datetime.timezone(datetime.timedelta(seconds=37800), 'ACDT'))

In [76]:
i = str(out)
i = i[:-6]
i

'2021-11-08 09:38:00'

In [74]:
#from datetime import timedelta
#out - timedelta(hours=9, minutes=30)

datetime.datetime(2021, 9, 24, 1, 59, tzinfo=datetime.timezone(datetime.timedelta(seconds=34200), 'ACST'))

In [77]:
from datetime import datetime   
import pytz

local = pytz.timezone("Australia/Adelaide")
naive = datetime.strptime(i, "%Y-%m-%d %H:%M:%S")
local_dt = local.localize(naive, is_dst=None)
utc_dt = local_dt.astimezone(pytz.utc)
utc_dt

datetime.datetime(2021, 11, 7, 23, 8, tzinfo=<UTC>)

In [78]:
utc_dt = utc_dt.replace(tzinfo=None)
utc_dt

datetime.datetime(2021, 11, 7, 23, 8)

In [79]:
unix_time = (utc_dt - datetime(1970,1,1)).total_seconds()
unix_time

1636326480.0

In [80]:
import googlemaps
from datetime import datetime

maps = googlemaps.Client(key=api_key)

now = datetime.now()
directions_result = maps.directions(source,
                                 destination,
                                 mode="driving",
                                avoid="ferries",
                                departure_time=unix_time,
                                traffic_model = "pessimistic")
print("The distance between points A & B is: ", directions_result[0]['legs'][0]['distance']['text'])
print("The normal time interval is: ", directions_result[0]['legs'][0]['duration']['text'])
print("The estimated time of arrival during peak hours is: ", directions_result[0]['legs'][0]['duration_in_traffic']['text'])


The distance between points A & B is:  14.6 km
The normal time interval is:  22 mins
The estimated time of arrival during peak hours is:  31 mins


In [81]:
# Extracting all the coordinate changes of the route from Source to Destination

start_cords = []
end_cords = []

# Looping through the directions_result dictionary and fetching all the start and end coordinates
for i in range(0,len(directions_result[0]['legs'][0]['steps'])):
    start_cords.append(directions_result[0]['legs'][0]['steps'][i]['start_location'])
    end_cords.append(directions_result[0]['legs'][0]['steps'][i]['end_location'])

start_cords = pd.DataFrame(start_cords)
end_cords = pd.DataFrame(end_cords)

all_cords = pd.concat([start_cords, end_cords[-1:]])

start_cords["start_lat_lng"] = start_cords["lat"].astype(str) + ', ' + start_cords["lng"].astype(str)
end_cords["end_lat_lng"] = end_cords["lat"].astype(str) + ', ' + end_cords["lng"].astype(str)

# Storing the coordinates inside a dataframe
start_end_cords = pd.concat([start_cords["start_lat_lng"], end_cords["end_lat_lng"]], axis = 1)

all_start_end_cords = pd.concat([start_cords["start_lat_lng"], end_cords["end_lat_lng"][-1:]])
all_start_end_cords = all_start_end_cords.reset_index(drop = True)

print(all_cords)
    
#print(end_cords["end_lat_lng"][-1:]) # printing only the last record of a dataframe as a dataframe
    #print("Start Location "+str(i),directions_result[0]['legs'][0]['steps'][i]['start_location'])
    #print("End Location "+str(i),directions_result[0]['legs'][0]['steps'][i]['end_location'])

         lat         lng
0 -35.005236  138.557838
1 -34.993740  138.556901
2 -34.996711  138.530533
3 -34.985050  138.520612
4 -34.944910  138.517650
5 -34.945433  138.504639
6 -34.937642  138.501814
7 -34.938106  138.500097
8 -34.925580  138.496010
8 -34.919530  138.494280


In [82]:
print(all_start_end_cords)

sites_cords = pd.DataFrame()

sites_cords["sites_cords"] = sites_lat_long["latitude"].astype(str) + ', ' + sites_lat_long["longitude"].astype(str)
print(sites_cords)

0    -35.0052358, 138.5578379
1      -34.99374, 138.5569013
2    -34.9967113, 138.5305329
3      -34.98505, 138.5206119
4    -34.9449099, 138.5176503
5    -34.9454333, 138.5046389
6    -34.9376422, 138.5018136
7    -34.9381061, 138.5000974
8    -34.9255802, 138.4960097
9    -34.9195303, 138.4942798
dtype: object
                 sites_cords
0     -34.986657, 138.523289
1     -34.872397, 138.548194
2      -34.838911, 138.48399
3     -34.844355, 138.505376
4     -34.852355, 138.509519
...                      ...
1184  -34.929844, 138.455414
1185  -34.997345, 138.630646
1186  -35.004837, 138.625351
1187  -34.929844, 138.455414
1188  -34.929844, 138.455414

[1189 rows x 1 columns]


In [83]:
sites_layer = gmaps.symbol_layer(
all_cords, fill_color='red', stroke_color='red'
)

# This code plots all the sites as a bubble plot (However, it is too slow as there are more than 1k points)
fig.add_layer(sites_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [60]:
#Create the map
#fig = gmaps.figure()
#create the layer
#layer1 = gmaps.directions.Directions((-34.8389048, 138.4839665), (-34.8390558, 138.5021301), mode='car')
#layer2 = gmaps.directions.Directions((-34.8390558, 138.5021301), (-34.837939, 138.5028254), mode='car')
#layer3 = gmaps.directions.Directions((-34.837939, 138.5028254), (-34.8034296, 138.6046136), mode='car')
#layer4 = gmaps.directions.Directions((-34.8034296, 138.6046136), (-34.8013831, 138.6064409), mode='car')
#layer5 = gmaps.directions.Directions((-34.8013831, 138.6064409), (-34.8090307, 138.631388), mode='car')
#layer6 = gmaps.directions.Directions((-34.8090307, 138.631388), (-34.81522229999999, 138.6417416), mode='car')
#layer7 = gmaps.directions.Directions((-34.81522229999999, 138.6417416), (-34.8241442, 138.6359468), mode='car')
#layer8 = gmaps.directions.Directions((-34.8241442, 138.6359468), (-34.8295101, 138.6615781), mode='car')
#layer9 = gmaps.directions.Directions((-34.8295101, 138.6615781), (-34.8297288, 138.6616282), mode='car')
#layer = gmaps.directions.Directions(source, destination, waypoints=[montreux], mode='car')
#Add the layer
#fig.add_layer(layer)
#fig.add_layer(layer1)
#fig.add_layer(layer2)
#fig.add_layer(layer3)
#fig.add_layer(layer4)
#fig.add_layer(layer5)
#fig.add_layer(layer6)
#fig.add_layer(layer7)
#fig.add_layer(layer8)
#fig.add_layer(layer9)
#fig


Figure(layout=FigureLayout(height='420px'))

In [65]:
#print(now)

In [129]:
#import haversine as hs
#loc1=(-34.8389048, 138.4839665)
#loc2=(-34.8390558, 138.5021301)
#hs.haversine(loc1,loc2)

1.6577779306719065

In [84]:
# Extracting the Distance Matrix of all the Nodes and the Coordinates of the Chosen Path using the 'Haversine Formula'

import haversine as hs

a = []
c = []
d = pd.DataFrame()

for i in range(0, len(sites_cords)):
    loc1 = tuple(map(float, sites_cords['sites_cords'][i].split(', ')))
    c.append(sites_cords['sites_cords'][i])

    for j in range(0, len(all_start_end_cords)):
        loc2 = tuple(map(float, all_start_end_cords[j].split(', ')))
        b = hs.haversine(loc1,loc2)
        a.append(b)
    
    a = pd.DataFrame(a)
    a = a.T
    d = pd.concat([d,a])
    a = []
    
d = d.reset_index(drop = True)
d.index = sites['Source']
d

,0,1,2,3,4,5,6,7,8,9
Source,,,,,,,,,,
2,3.764551,3.161636,1.298208,0.302343,4.670422,4.888769,5.790907,5.797546,7.232169,7.918556
3,14.797156,13.516072,13.916577,12.776345,8.530565,9.040420,8.397785,8.521864,7.590851,7.186469
4,19.681925,18.455179,18.052486,16.589499,12.179815,11.993541,11.098137,11.127419,9.699355,9.013476
6,18.517507,17.262417,17.095833,15.706151,11.237110,11.239611,10.378165,10.435772,9.072159,8.420179
7,17.561037,16.304028,16.165636,14.789655,10.318331,10.359414,9.509519,9.573739,8.234986,7.597795
...,...,...,...,...,...,...,...,...,...,...
20967,12.545269,11.662415,10.106551,8.543209,5.915348,4.810292,4.317726,4.175618,3.731195,3.724341
20968,6.689463,6.729401,9.119473,10.116412,11.832200,12.850816,13.486524,13.597774,14.635989,15.142844
20969,6.149258,6.355772,8.683624,9.791232,11.861641,12.829571,13.510521,13.612869,14.716710,15.252627


In [85]:
a = []
e = pd.DataFrame()

for i in range(0,len(d)):
    for j in range(0, len(d.T)):
        if(d.iloc[i,j] <= 0.5):
            a.append(d.index[i])

a = list(set(a))
a

for i in range(0, len(a)):
    b = a[i]
    c = sites[['latitude','longitude']][sites['Source'] == b]
    #c = sites[['Source','latitude','longitude']][sites['Source'] == b] # Check whether the Source Ids are mathhing between a & e
    e = pd.concat([e, c])
    

In [220]:
#sites[['Source','latitude','longitude']][sites['Source'] == 20887]

pandas.core.frame.DataFrame

In [86]:
sites_layer = gmaps.symbol_layer(
e, fill_color='green', stroke_color='green'
)

# This code plots all the sites as a bubble plot (However, it is too slow as there are more than 1k points)
fig.add_layer(sites_layer)
fig.add_layer(gmaps.traffic_layer()) # Adds information related to the current state of traffic
fig

Figure(layout=FigureLayout(height='420px'))